In [1]:
# source dataset : https://www.imdb.com/interfaces/

In [2]:
#lessons I learned:
#https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
#https://stackoverflow.com/questions/31617845/how-to-select-rows-in-a-dataframe-between-two-values-in-python-pandas/40442778
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
#https://stackoverflow.com/questions/36370839/better-way-to-drop-nan-rows-in-pandas
#https://cmdlinetips.com/2018/01/how-to-get-unique-values-from-a-column-in-pandas-data-frame/
#https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning/15778297
#https://pbpython.com/pandas_dtypes.html
#https://www.ritchieng.com/pandas-exploring-series/
#https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html


In [3]:
import pandas as pd
import csv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from functools import reduce

In [37]:
ratings = pd.DataFrame.from_csv('title.ratings.tsv', sep='\t')

allmovies = pd.DataFrame.from_csv('title.basics.tsv', sep='\t')

languagemovies = pd.DataFrame.from_csv('title.akas.tsv', sep='\t')


/Users/chrischung/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/Users/chrischung/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [5]:
print(allmovies.shape)
print(languagemovies.shape)
print(ratings.shape)

(5974826, 8)
(3683865, 7)
(947891, 2)


In [41]:
#filter movies all in USA, I'm assuming it means the main language is English
englishmovies = languagemovies[languagemovies['region'] == 'US']
columns = ['titleId', 'title', 'region']
englishmovies['titleId'] = englishmovies.index
englishmovies = englishmovies[columns]
englishmovies.head()       

,titleId,title,region
titleId,,,
tt0000001,tt0000001,Carmencita,US
tt0000002,tt0000002,The Clown and His Dogs,US
tt0000005,tt0000005,Blacksmithing Scene,US
tt0000005,tt0000005,Blacksmith Scene #1,US
tt0000005,tt0000005,Blacksmithing,US


In [44]:
#create titleId column to join tables later on
allmovies['titleId'] = allmovies.index
#get a titleId to prep for a merge on titleId
ratings['titleId'] = ratings.index

#movies only
allmovies = allmovies.loc[allmovies['titleType'] == 'movie']
allmovies.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId
tconst,,,,,,,,,
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,tt0000009
tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport",tt0000147
tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama",tt0000335
tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,tt0000502
tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama",tt0000574


In [46]:
#combine ALL matching on titleId, forget everything else.
results = ratings.merge(englishmovies, how='inner', on='titleId').merge(allmovies, how='inner', on='titleId')
results.head()

,averageRating,numVotes,titleId,title,region,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,5.5,80,tt0000009,Miss Jerry,US,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,5.2,300,tt0000147,The Corbett-Fitzsimmons Fight,US,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
2,2.9,11,tt0000630,Hamlet,US,movie,Hamlet,Amleto,0,1908,\N,\N,Drama
3,4.8,33,tt0000679,The Fairylogue and Radio-Plays,US,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,\N,120,"Adventure,Fantasy"
4,5.1,19,tt0000886,"Hamlet, Prince of Denmark",US,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,\N,Drama


In [47]:
print(results.shape)

(111889, 13)


In [49]:
#remove columns i'm not using and create new table
columns = ['primaryTitle', 'startYear', 'genres', 'averageRating', 'numVotes', 'runtimeMinutes', 'titleType', 'region', 'titleId']
results = results[columns]
results.head()

,primaryTitle,startYear,genres,averageRating,numVotes,runtimeMinutes,titleType,region,titleId
0,Miss Jerry,1894,Romance,5.5,80,45,movie,US,tt0000009
1,The Corbett-Fitzsimmons Fight,1897,"Documentary,News,Sport",5.2,300,20,movie,US,tt0000147
2,Hamlet,1908,Drama,2.9,11,\N,movie,US,tt0000630
3,The Fairylogue and Radio-Plays,1908,"Adventure,Fantasy",4.8,33,120,movie,US,tt0000679
4,"Hamlet, Prince of Denmark",1910,Drama,5.1,19,\N,movie,US,tt0000886


In [50]:
#turn start year and runtime Minutes into integer and remove NA
results = results[results['startYear'] != '\\N']
results = results[results['runtimeMinutes'] != '\\N']

#turn start year and runtime Minutes into integers from strings
results['startYear'] = results['startYear'].astype('int')
results['runtimeMinutes'] = results['runtimeMinutes'].astype('int')

#Keeping only movies that started from 1990 up to 2019
results = results.loc[(results['startYear'] >= 1990) & (results['startYear'] <= 2019)]

results.head()

,primaryTitle,startYear,genres,averageRating,numVotes,runtimeMinutes,titleType,region,titleId
12797,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,74330,118,movie,US,tt0035423
12798,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,74330,118,movie,US,tt0035423
26357,"Wenn du groß bist, lieber Adam",1990,"Drama,Fantasy",6.9,23,78,movie,US,tt0059900
30878,The Other Side of the Wind,2018,Drama,6.9,4589,122,movie,US,tt0069049
36244,Me and the Kid,1993,"Comedy,Crime,Drama",5.3,211,94,movie,US,tt0081145


In [51]:
#Let's remove duplicates with same titleId because same title doesn't necessarily mean same movie 
#There are a lot of movies that are listed as different names
#There's something called 'working title' which means it was a tentative title used during development
results[results['titleId'] == results['titleId']].head()

,primaryTitle,startYear,genres,averageRating,numVotes,runtimeMinutes,titleType,region,titleId
12797,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,74330,118,movie,US,tt0035423
12798,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,74330,118,movie,US,tt0035423
26357,"Wenn du groß bist, lieber Adam",1990,"Drama,Fantasy",6.9,23,78,movie,US,tt0059900
30878,The Other Side of the Wind,2018,Drama,6.9,4589,122,movie,US,tt0069049
36244,Me and the Kid,1993,"Comedy,Crime,Drama",5.3,211,94,movie,US,tt0081145


In [52]:
#Drop duplicate under titleId
results.drop_duplicates(keep='first',subset='titleId',inplace=True)

In [57]:
print(results.shape)
results.head()

(45924, 9)


,primaryTitle,startYear,genres,averageRating,numVotes,runtimeMinutes,titleType,region,titleId
12797,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,74330,118,movie,US,tt0035423
26357,"Wenn du groß bist, lieber Adam",1990,"Drama,Fantasy",6.9,23,78,movie,US,tt0059900
30878,The Other Side of the Wind,2018,Drama,6.9,4589,122,movie,US,tt0069049
36244,Me and the Kid,1993,"Comedy,Crime,Drama",5.3,211,94,movie,US,tt0081145
36467,Vincent and Me,1990,"Drama,Family,Fantasy",6.5,278,100,movie,US,tt0081721


In [58]:
#save our dataframe to a pickle for use in other notebooks
results.to_pickle('results.pkl')